In [1]:
"""Supply and discharge data is provided as m3second. Convert to millionm3.
-------------------------------------------------------------------------------
PCRGLOBWB Data for demand is provided in volumes with units m3second
(implicit per pixel/per time step); converting to volumes (millionm3) and 
storing to 
earth engine asset. 


Author: Rutger Hofste
Date: 20180419
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    SCRIPT_NAME (string) : Script name
    EE_VERSION (integer) : Earth Engine path version.
    OUTPUT_VERSION (integer) : Output version.
    TESTING (boolean) : Testing.

Returns:

"""

# Input Parameters

SCRIPT_NAME = "Y2018M04D19_RH_Supply_Discharge_Volume_5min_EE_V01"
EE_VERSION = 9

OUTPUT_VERSION = 1 

TESTING = 0

SEPARATOR = "_|-"

SCHEMA = ["geographic_range",
     "temporal_range",
     "indicator",
     "temporal_resolution",
     "unit",
     "spatial_resolution",
     "temporal_range_min",
     "temporal_range_max"]

ee_path = "projects/WRI-Aquaduct/PCRGlobWB20V{:02.0f}".format(EE_VERSION)


print("Input ee: " + ee_path +
      "\nOutput ee: " + ee_path )

Input ee: projects/WRI-Aquaduct/PCRGlobWB20V09
Output ee: projects/WRI-Aquaduct/PCRGlobWB20V09


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M04D24 UTC 13:08


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import subprocess
import pandas as pd
import ee
import re
from retrying import retry
from datetime import timedelta
import aqueduct3

ee.Initialize()

In [4]:
def update_property_script_used(image):
    image = image.set("script_used",SCRIPT_NAME)
    return image

def update_property_output_version(image):
    image = image.set("output_version",OUTPUT_VERSION)
    return image


def ic_flux_to_volume_5min_m3second_millionm3(ic_input_asset_id,output_version,old_units,new_units,schema,separator):
    """ Convert an imagecollection from flux to volume.
    -------------------------------------------------------------------------------
    The result is stored in an imagecollection with the same name as the input
    imagecollection but with 'millionm3' instead of 'm3second'
    
    Input ic:
    global_historical_riverdischarge_month_m3second_5min_1960_2014
    
    Output ic:    
    global_historical_riverdischarge_month_millionm3_5min_1960_2014
    
    Args:
        ic_input_asset_id (string) : asset id of input imagecollection.
        output_version (integer) : output version. 
        old_units (string) : Old units.
        new_units (string) : New units. 
        schema (list) : A list of strings containing the schema. See 
            aqueduct3.split_key() for more info.
        separator (regex) : Regular expression of separators used in geotiff
            filenames.    

    """
    start_time = time.time()
    df_errors = pd.DataFrame()
    
    df = aqueduct3.earthengine.get_df_from_ic(ic_input_asset_id)
    df = aqueduct3.earthengine.add_export_parameters_for_export(df,old_unit,new_unit,output_version)
    
    # Creating ImageCollection(s)
    output_ic_asset_ids = list(df["output_ic_asset_id"].unique())
    for output_ic_asset_id in output_ic_asset_ids:
        command, result = aqueduct3.earthengine.create_imageCollection(output_ic_asset_id)
        print(command,result)
    
    # Bacth Converting and uploading.     
    for index, row in df.iterrows():
        elapsed_time = time.time() - start_time 
        print("Index: {:04.0f} Elapsed: {}".format(index, timedelta(seconds=elapsed_time)))
    
        description = row["description"]
        output_image_asset_id = row["output_image_asset_id"]
        
        # get additional parameters from asset name
        # add fictional extension to 
        key = row["input_image_asset_id"] + ".ee_image"
        dictje = aqueduct3.split_key(key,schema,separator)
        
        
    
        if aqueduct3.earthengine.asset_exists(output_image_asset_id):
            print("Asset exists, skipping: {}".format(output_image_asset_id))
        else:
            i_old_unit_5min = ee.Image(row["input_image_asset_id"])                
            if old_unit == "m3second" and new_unit == "millionm3":
                year = int(dictje["year"])
                month = int(dictje["month"])
                temporal_resolution = dictje["temporal_resolution"]
                
                i_new_unit_5min = aqueduct3.earthengine.flux_to_volume_5min_m3second_millionm3(i_old_unit_5min,temporal_resolution,year,month) 
            else:
                raise("Error: invalid combination of units.") 
            i_new_unit_5min = update_property_script_used(i_new_unit_5min)
            i_new_unit_5min = update_property_output_version(i_new_unit_5min)
            

            aqueduct3.earthengine.export_image_global_5min(i_new_unit_5min,description,output_image_asset_id)
            print(output_image_asset_id)  
         

    return i_new_unit_5min


In [5]:
old_unit = "m3second"
new_unit = "millionm3"

temporal_resolutions = ["year","month"]
indicators = ["riverdischarge"]

In [6]:
df = pd.DataFrame()
for indicator in indicators:
    for temporal_resolution in temporal_resolutions:
        ic_input_file_name = "global_historical_{}_{}_m3second_5min_1960_2014".format(indicator,temporal_resolution)
        ic_input_asset_id = "{}/{}".format(ee_path,ic_input_file_name)
        df = ic_flux_to_volume_5min_m3second_millionm3(ic_input_asset_id,OUTPUT_VERSION,"m3second","millionm3",SCHEMA,SEPARATOR)

earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014 b'Asset projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014 already exists\n'
Index: 0000 Elapsed: 0:00:04.605320
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I000Y1960M01
Index: 0001 Elapsed: 0:00:08.795913
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I001Y1961M01
Index: 0002 Elapsed: 0:00:10.964594
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I002Y1962M01
Index: 0003 Elapsed: 0:00:14.944947
projects/WRI-Aquaduct/PCRGlobWB20V09/global_hist

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I038Y1998M01
Index: 0039 Elapsed: 0:02:26.255851
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I039Y1999M01
Index: 0040 Elapsed: 0:02:29.427159
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I040Y2000M01
Index: 0041 Elapsed: 0:02:33.076444
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min_1960_2014_I041Y2001M01
Index: 0042 Elapsed: 0:02:36.679477
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_millionm3_5min_1960_2014/global_historical_riverdischarge_year_millionm3_5min

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I021Y1961M10
Index: 0022 Elapsed: 0:01:22.902025
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I022Y1961M11
Index: 0023 Elapsed: 0:01:26.676877
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I023Y1961M12
Index: 0024 Elapsed: 0:01:30.405824
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I024Y1962M01
Index: 0025 Elapsed: 0:01:34.118067
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I060Y1965M01
Index: 0061 Elapsed: 0:03:47.142937
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I061Y1965M02
Index: 0062 Elapsed: 0:03:50.310934
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I062Y1965M03
Index: 0063 Elapsed: 0:03:53.949374
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I063Y1965M04
Index: 0064 Elapsed: 0:03:57.614467
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I099Y1968M04
Index: 0100 Elapsed: 0:06:08.432225
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I100Y1968M05
Index: 0101 Elapsed: 0:06:11.641827
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I101Y1968M06
Index: 0102 Elapsed: 0:06:15.325195
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I102Y1968M07
Index: 0103 Elapsed: 0:06:19.005303
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

...

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I489Y2000M10
Index: 0490 Elapsed: 0:29:29.984189
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I490Y2000M11
Index: 0491 Elapsed: 0:29:33.652146
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I491Y2000M12
Index: 0492 Elapsed: 0:29:36.832954
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I492Y2001M01
Index: 0493 Elapsed: 0:29:40.608483
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I528Y2004M01
Index: 0529 Elapsed: 0:31:49.635578
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I529Y2004M02
Index: 0530 Elapsed: 0:31:53.327442
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I530Y2004M03
Index: 0531 Elapsed: 0:31:57.036355
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I531Y2004M04
Index: 0532 Elapsed: 0:32:00.740366
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I567Y2007M04
Index: 0568 Elapsed: 0:34:09.987303
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I568Y2007M05
Index: 0569 Elapsed: 0:34:13.834889
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I569Y2007M06
Index: 0570 Elapsed: 0:34:17.711100
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I570Y2007M07
Index: 0571 Elapsed: 0:34:20.890103
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I606Y2010M07
Index: 0607 Elapsed: 0:36:32.011260
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I607Y2010M08
Index: 0608 Elapsed: 0:36:35.649603
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I608Y2010M09
Index: 0609 Elapsed: 0:36:39.540887
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I609Y2010M10
Index: 0610 Elapsed: 0:36:42.261227
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I645Y2013M10
Index: 0646 Elapsed: 0:38:49.402971
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I646Y2013M11
Index: 0647 Elapsed: 0:38:53.037138
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I647Y2013M12
Index: 0648 Elapsed: 0:38:56.626720
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_millionm3_5min_1960_2014_I648Y2014M01
Index: 0649 Elapsed: 0:39:00.468567
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_millionm3_5min_1960_2014/global_historical_riverdischarge_month_mill

In [7]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:43:08.519624


Previous runs:  
0:43:08.519624